## Explication type (1-1)

In [2]:
from gurobipy import *

# Données du problème
data = {
    'A': 32,
    'B': 0,
    'C': -28,
    'D': 36,
    'E': 48,
    'F': -35,
    'G': -42
}

# Identification des ensembles Pros et Cons
pros = [k for k, v in data.items() if v > 0]  # Ensemble Pros(x,y)
cons = [k for k, v in data.items() if v < 0]  # Ensemble Cons(x,y)

print(f"Pros: {pros}")
print(f"Cons: {cons}")

# Calcul des Trade-offs (1-1) valides
valid_pairs = []
for p in pros:
    for q in cons:
        if data[p] + data[q] > 0:
            valid_pairs.append((p, q))

print(f"Nombre de paires valides identifiées : {len(valid_pairs)}")

# Modélisation avec Gurobi

# Instanciation du modèle
m = Model("Explication_TradeOff_1_1")

# Création des variables binaires c[(p,q)]
# c = 1 si la paire est choisie, 0 sinon
c = {}
for p, q in valid_pairs:
    c[(p, q)] = m.addVar(vtype=GRB.BINARY, name=f"c_{p}_{q}")

# Mise à jour du modèle
m.update()

# Ajout des contraintes

# Contrainte 1 : Tout élément de Cons doit être couvert exactement une fois
# Somme(c_pq) = 1 pour chaque q dans Cons
for q in cons:
    # On utilise quicksum pour sommer efficacement les variables Gurobi concernées
    m.addConstr(quicksum(c[(p, q)] for p in pros if (p, q) in valid_pairs) == 1, name=f"Cover_{q}")

# Contrainte 2 : Tout élément de Pros est utilisé au maximum une fois
# Les trade-offs doivent être disjoints
for p in pros:
    m.addConstr(quicksum(c[(p, q)] for q in cons if (p, q) in valid_pairs) <= 1, name=f"Unique_{p}")

# Fonction Objectif
# Ici on cherche juste la faisabilité, on peut minimiser 0
m.setObjective(0, GRB.MINIMIZE)

# Paramétrage (mode silencieux)
m.params.outputflag = 0

# Résolution du PL
m.optimize()

# --- 4. Analyse des résultats ---
if m.status == GRB.OPTIMAL:
    print("\nSolution Optimale Trouvée (Explication Valide) :")
    explanation = []
    for p, q in valid_pairs:
        # On vérifie si la variable est à 1 (avec une petite tolérance pour les flottants)
        if c[(p, q)].x > 0.5:
            explanation.append((p,q))
            net_val = data[p] + data[q]
            print(f" - Trade-off ({p},{q})")
    
    # Vérification
    expected = {('A', 'C'), ('D', 'F'), ('E', 'G')}
    is_same = set(explanation) == expected

elif m.status == GRB.INFEASIBLE:
    print("\nModèle Infeasible : Il n'existe pas d'explication de type (1-1).")
    # Certificat de non-existence

Pros: ['A', 'D', 'E']
Cons: ['C', 'F', 'G']
Nombre de paires valides identifiées : 6
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2755100
Academic license 2755100 - for non-commercial use only - registered to na___@student-cs.fr

Solution Optimale Trouvée (Explication Valide) :
 - Trade-off (A,C)
 - Trade-off (D,F)
 - Trade-off (E,G)


## Explication type (1-m)

In [3]:
from gurobipy import *

# DONNÉES DU PROBLÈME
weights = {'A': 8, 'B': 7, 'C': 7, 'D': 6, 'E': 6, 'F': 5, 'G': 6}

# Notes des candidats u et v
notes_u = {'A': 72, 'B': 66, 'C': 75, 'D': 85, 'E': 88, 'F': 66, 'G': 93}
notes_v = {'A': 71, 'B': 73, 'C': 63, 'D': 92, 'E': 76, 'F': 79, 'G': 93}

# Calcul des contributions (deltas)
deltas = {}
for k in weights:
    deltas[k] = weights[k] * (notes_u[k] - notes_v[k])

pros = [k for k, v in deltas.items() if v > 0]
cons = [k for k, v in deltas.items() if v < 0]

print(f"Arguments Pour (Pros) : {pros}")
print(f"Arguments Contre (Cons) : {cons}")

# MODÉLISATION GUROBI
m = Model("Explication_1_m")

# Variables de décision UNIQUEMENT x
# x[p, c] = 1 si l'argument Pro 'p' couvre l'argument Con 'c'
x = m.addVars(pros, cons, vtype=GRB.BINARY, name="x")

m.update()

# CONTRAINTES

# A. Couverture complète : Chaque Con doit être couvert par exactement un Pro
for c in cons:
    m.addConstr(quicksum(x[p, c] for p in pros) == 1, name=f"Cover_{c}")

# B. Validité du Trade-off
# Formule : Delta_p + Somme(Delta_c * x_pc) >= epsilon
# Si p est utilisé, cela vérifie que le trade-off est positif.
# Si p est INUTILISÉ (x_pc = 0 partout), cela devient Delta_p >= epsilon, ce qui est toujours VRAI.
epsilon = 0.001 

for p in pros:
    # Note: deltas[c] est négatif ici
    m.addConstr(deltas[p] + quicksum(deltas[c] * x[p, c] for c in cons) >= epsilon, name=f"Valid_{p}")

# RÉSOLUTION
m.params.outputflag = 0  
m.optimize()

# ANALYSE
print("-" * 30)
if m.status == GRB.OPTIMAL:
    print("Résultat : Une explication de type (1-m) existe !\n")
    
    # On itère sur les Pros et on vérifie s'ils ont des Cons assignés
    for p in pros:
        covered_cons = [c for c in cons if x[p, c].x > 0.5]
        
        # Si la liste n'est pas vide, c'est que ce Pro fait partie de l'explication
        if covered_cons:
            net_value = deltas[p] + sum(deltas[c] for c in covered_cons)
            print(f"Groupe mené par {p} (+{deltas[p]}) :")
            print(f"  -> Couvre : {covered_cons} (Poids cumulé Cons: {sum(deltas[c] for c in covered_cons)})")
            print(f"  -> Bilan net : +{net_value}")

elif m.status == GRB.INFEASIBLE:
    print("Résultat : INFEASIBLE (Certificat de non-existence).")

Arguments Pour (Pros) : ['A', 'C', 'E']
Arguments Contre (Cons) : ['B', 'D', 'F']
------------------------------
Résultat : INFEASIBLE (Certificat de non-existence).


# Application pour le dataset RATP

## Réutilisation du code précédent

In [11]:
from gurobipy import *
import pandas as pd

# DONNÉES DU PROBLÈME
df_weights = pd.read_excel(
    "RATP.xlsx",
    usecols="C:I",
    skiprows=13,
    nrows=1,
) # type: ignore

# Notes des candidats u et v
data = pd.read_excel(
    "RATP.xlsx",
    usecols="B:I",
    skiprows=2,
    nrows=10,
    index_col=0,
)
df_weights.columns = data.columns
weights = df_weights.iloc[0].to_dict()

# Pour chaque paire (u,v) on effectue l'analyse précédente

for u in data.index:
    for v in data.index:
        if u == v:
            continue
        
        print(f"\n\n--- Analyse de l'explication entre {u} et {v} ---")

        # Calcul des contributions (deltas)
        deltas = {}
        for k in weights:
            deltas[k] = weights[k] * (data.loc[u, k] - data.loc[v, k])  

        pros = [k for k, v in deltas.items() if v > 0]
        cons = [k for k, v in deltas.items() if v < 0]

        print(f"Arguments Pour (Pros) : {pros}")
        print(f"Arguments Contre (Cons) : {cons}")

        # MODÉLISATION GUROBI
        m = Model("Explication_1_m")

        # Variables de décision UNIQUEMENT x
        # x[p, c] = 1 si l'argument Pro 'p' couvre l'argument Con 'c'
        x = m.addVars(pros, cons, vtype=GRB.BINARY, name="x")

        m.update()

        # CONTRAINTES

        # A. Couverture complète : Chaque Con doit être couvert par exactement un Pro
        for c in cons:
            m.addConstr(quicksum(x[p, c] for p in pros) == 1, name=f"Cover_{c}")

        # B. Validité du Trade-off
        # Formule : Delta_p + Somme(Delta_c * x_pc) >= epsilon
        # Si p est utilisé, cela vérifie que le trade-off est positif.
        # Si p est INUTILISÉ (x_pc = 0 partout), cela devient Delta_p >= epsilon, ce qui est toujours VRAI.
        epsilon = 0.001 

        for p in pros:
            # Note: deltas[c] est négatif ici
            m.addConstr(deltas[p] + quicksum(deltas[c] * x[p, c] for c in cons) >= epsilon, name=f"Valid_{p}")

        # RÉSOLUTION
        m.params.outputflag = 0  
        m.optimize()

        # ANALYSE
        print("-" * 30)
        if m.status == GRB.OPTIMAL:
            print("Résultat : Une explication de type (1-m) existe !\n")
            
            # On itère sur les Pros et on vérifie s'ils ont des Cons assignés
            for p in pros:
                covered_cons = [c for c in cons if x[p, c].x > 0.5]
                
                # Si la liste n'est pas vide, c'est que ce Pro fait partie de l'explication
                if covered_cons:
                    net_value = deltas[p] + sum(deltas[c] for c in covered_cons)
                    print(f"Groupe mené par {p} (+{deltas[p]}) :")
                    print(f"  -> Couvre : {covered_cons} (Poids cumulé Cons: {sum(deltas[c] for c in covered_cons)})")
                    print(f"  -> Bilan net : +{net_value}")

        elif m.status == GRB.INFEASIBLE:
            print("Résultat : INFEASIBLE (Certificat de non-existence).")



--- Analyse de l'explication entre Odéon (Ligne 4) et Place d'Italie (Lign 6) ---
Arguments Pour (Pros) : ['peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]']
Arguments Contre (Cons) : ['off-peak-entering-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]']
------------------------------
Résultat : Une explication de type (1-m) existe !

Groupe mené par peak-entering-passengers/h (+85.99376545200472) :
  -> Couvre : ['off-peak-entering-passengers/h'] (Poids cumulé Cons: -75.24454477050415)
  -> Bilan net : +10.749220681500574
Groupe mené par off-peak-passing-passengers/h (+96.7429861335053) :
  -> Couvre : ['Station degradation level ([0,20]  scale)'] (Poids cumulé Cons: -94.0556809631303)
  -> Bilan net : +2.687305170374998
Groupe mené par strategic priority [0,10] (+128.9906481780071) :
  -> Couvre : ['connectivity index [0,100]'] (Poids cumulé Cons: -112.86681715575621)
  -> Bilan net : +16.12383102225088